<a href="https://colab.research.google.com/github/jacob9301/Assignments/blob/main/DataLinkage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fuzzymatcher
!pip install recordlinkage

     |████████████████████████████████| 50 kB 3.7 MB/s 
  Created wheel for metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13919 sha256=2c76d8aa0d1e5bf2b09ecdc02c650586955ae168f87216835157ec40abd0c79d
  Stored in directory: /root/.cache/pip/wheels/1d/a8/cb/6f8902aa5457bd71344e00665c230e9c45255b3f57f2194a0f
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149859 sha256=4221458e1472d3d4878d17298d88b8e30f734ab8eabdfc667bbfefc240ca742f
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built metaphone python-Levenshtein
     |████████████████████████████████| 944 kB 8.3 MB/s 
     |████████████████████████████████| 137 kB 39.9 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.8.9-cp37-cp37m-linux_x86_64.whl size=73216 sha256=871339f1cdc9288995339ec43d4462252ec54689bfc441683a3fe6bd0e5161a6
  Stored in directory: /root/.cache/pip/wheels/32/a9/ef/5d8

In [ ]:
import pandas as pd
import fuzzymatcher
import recordlinkage

In [ ]:
ACM = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv')
DBLP = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv', 
                   encoding='latin_1')
groundTruth = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/DBLP-ACM_perfectMapping.csv')

#The Data Sets
both data sets provide the same columns. However, different naming convensions have been used for id in the two data sets. title, authors and venue also have inconsistencies between the datasets. Fore example, this could be differences in capitalisation, order of names or the use of acronyms. The year column in both data sets follow the same convension (YYYY).

In [ ]:
ACM.head(10)

,Unnamed: 0,id,title,authors,venue,year
0,0,304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,1,304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,2,304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999
3,3,304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",International Conference on Management of Data,1999
4,4,304582,The CCUBE constraint object-oriented database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",International Conference on Management of Data,1999
5,5,304583,The Cornell Jaguar project: adding mobility to...,"Phillippe Bonnet, Kyle Buza, Zhiyuan Chan, Vic...",International Conference on Management of Data,1999
6,6,304584,The active MultiSync controller of the cubetre...,"Nick Roussopoulos, Yannis Kotidis, Yannis Sism...",International Conference on Management of Data,1999
7,7,304585,The Jungle database search engine,"Michael B&#246;hlen, Linas Bukauskas, Curtis D...",International Conference on Management of Data,1999
8,8,306112,ADEPT: an agent-based approach to business pro...,"N. R. Jennings, T. J. Norman, P. Faratin",ACM SIGMOD Record,1998
9,9,306115,A componentized architecture for dynamic elect...,"Benny Reich, Israel Ben-Shaul",ACM SIGMOD Record,1998


In [ ]:
DBLP.head(10)

,Unnamed: 0,id,title,authors,venue,year
0,0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models f...,D. Scott Mackay,SIGMOD Record,1999
1,1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and it...,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive A...,"Themistoklis Palpanas, Richard Sidle, Hamid Pi...",VLDB,2002
3,3,conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Proces...,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert G...",VLDB,1996
4,4,conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spat...,"Erik G. Hoel, Hanan Samet",VLDB,1995
5,5,conf/sigmod/Keim99,Efficient Geometry-based Similarity Search of ...,Daniel A. Keim,SIGMOD Conference,1999
6,6,journals/sigmod/Ouksel02,Mining the World Wide Web: An Information Sear...,Aris M. Ouksel,SIGMOD Record,2002
7,7,journals/vldb/Seshadri98,Enhanced Abstract Data Types in Object-Relatio...,Praveen Seshadri,VLDB J.,1998
8,8,journals/sigmod/RamamrithamS97,Report on DART '96: Databases: Active and Real...,"Nandit Soparkar, Krithi Ramamritham",SIGMOD Record,1997
9,9,journals/sigmod/DAndreaJ96,UniSQL's Next-Generation Object-Relational Dat...,"Phil Janus, Albert D'Andrea",SIGMOD Record,1996


In [ ]:
print("raw dataframe number of records:")
print("ACM:  {x}".format(x=ACM['title'].count()))
print("DBLP:  {x}".format(x=DBLP['title'].count()))

raw dataframe number of records:
ACM:  2294
DBLP:  2616


# Record linking using using Fuzzymatcher

first identify the columns we want to match on

In [ ]:
# columns from the left dataframe
left_on = ["title", "authors", "venue", "year"]
# columns from the right dataframe
right_on = ["title", "authors", "venue", "year"]

Then perform the match

In [ ]:
matched_results = fuzzymatcher.fuzzy_left_join(ACM,
                                               DBLP,
                                               left_on,
                                               right_on,
                                               left_id_col='id',
                                               right_id_col='id')

Show the best matches

In [ ]:
cols = [
    "best_match_score", "title_left", "title_right", "authors_left", 
    "authors_right", "venue_left", "venue_right", "year_left", "year_right"
]
matched_results[cols].sort_values(by=['best_match_score'], 
                                  ascending=False).head(10)

,best_match_score,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right
29022,9.659057,TSQL2 language specification,TSQL2 Language Specification,"Richard Thomas Snodgrass, Ilsoo Ahn, Gadi Aria...","Wolfgang KÃ¤fer, Curtis E. Dyreson, Arie Segev...",ACM SIGMOD Record,SIGMOD Record,1994,1994
28358,9.585550,A TSQL2 tutorial,A TSQL2 Tutorial,"Richard Thomas Snodgrass, Ilsoo Ahn, Gad Ariav...","James Clifford, Ramez Elmasri, Michael D. Soo,...",ACM SIGMOD Record,SIGMOD Record,1994,1994
25021,9.299090,The Propel Distributed Services Platform,The Propel Distributed Services Platform,"Michael J. Carey, Steve Kirsch, Mary Roth, Ber...","Runping Qi, Kuan Yee, Brian Sterling, Nicolas ...",Very Large Data Bases,VLDB,2001,2001
34505,9.249386,A mapping mechanism to support bitmap index an...,A mapping mechanism to support bitmap index an...,"Eugene Inseok Chong, Jagannathan Srinivasan, S...","Mahesh Jagannath, Eugene Inseok Chong, Aravind...",ACM SIGMOD Record,SIGMOD Record,2003,2003
28308,8.833548,Announcement-the temporal query language TSQL2...,TSQL2 Language Specification,"Richard Thomas Snodgrass, Ilsoo Ahn, Gad Ariav...","Wolfgang KÃ¤fer, Curtis E. Dyreson, Arie Segev...",ACM SIGMOD Record,SIGMOD Record,1994,1994
20397,8.673152,XML Repository and Active Views Demonstration,XML Repository and Active Views Demonstration,"Serge Abiteboul, Vincent Aguilera, S&#233;bast...","Bruno Tessier, AmÃ©lie Marian, Brendan Hills, ...",Very Large Data Bases,VLDB,1999,1999
427,8.591388,PowerBookmarks: a system for personalizable We...,PowerBookmarks: A System for Personalizable We...,"Wen-Syan Li, Quoc Vu, Edward Chang, Divyakant ...","Kazuyuki Shimazu, Sougata Mukherjea, Yi-Leh Wu...",International Conference on Management of Data,SIGMOD Conference,1999,1999
7252,8.526615,Evolution and change in data management - issu...,Evolution and Change in Data Management - Issu...,"John F. Roddick, Lina Al-Jadir, Leopoldo Berto...","Kathleen Hornsby, Heidi Gregersen, Lex Wedemei...",ACM SIGMOD Record,SIGMOD Record,2000,2000
25114,7.772507,Oracle8i Index-Organized Table and Its Applica...,Oracle8i Index-Organized Table and Its Applica...,"Jagannathan Srinivasan, Souripriya Das, Chuck ...","Anh-Tuan Tran, Chuck Freiwald, Aravind Yalaman...",Very Large Data Bases,VLDB,2000,2000
784,7.639077,Chorochronos: a research network for spatiotem...,Chorochronos: A Research Network for Spatiotem...,"Andrew Frank, Stephane Grumbach, Ralf Hartmut ...","Enrico Nardelli, Timos K. Sellis, Andrew U. Fr...",ACM SIGMOD Record,SIGMOD Record,1999,1999


And the worst matches

In [ ]:
matched_results[cols].sort_values(
    by=['best_match_score'], ascending=True).head(10)

,best_match_score,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right
19451,-1.971479,David DeWitt speaks out: on rethinking the CS ...,Gio Wiederhold Speaks Out on Moving into Acade...,Marianne Winslett,Marianne Winslett,ACM SIGMOD Record,SIGMOD Record,2002,2001
5652,-1.279428,Challenges in automating declarative business ...,Expressing Business Rules,Val Huber,Ronald G. Ross,International Conference on Management of Data,SIGMOD Conference,2000,2000
33976,-1.272988,Abstracts of invited industrial track presenta...,Objectivity Industrial Exhibit,Zachary Ives,Objectivity,International Conference on Management of Data,VLDB,2003,1998
5408,-1.258954,Going beyond personalization: rule engines at ...,Delivering High Availability for Inktomi Searc...,Eric Kintzer,Eric A. Brewer,International Conference on Management of Data,SIGMOD Conference,2000,1998
15418,-1.204974,Special system-oriented section: the best of S...,A Pattern-Based Object Calculus,Stanley Y. W. Su,"Stanley Y. W. Su, Nabil Kamel, Ping Wu",The VLDB Journal &mdash; The International Jou...,VLDB J.,1995,1994
29136,-1.200692,A consensus glossary of temporal database conc...,A TSQL2 Tutorial,"Curtis Dyreson, Fabio Grandi, Wolfgang K&#228;...","James Clifford, Ramez Elmasri, Michael D. Soo,...",ACM SIGMOD Record,SIGMOD Record,1994,1994
20541,-1.124532,Hypothetical knowledge and counterfactual reas...,Online Query Processing,Joseph Y. Halpern,"Peter J. Haas, Joseph M. Hellerstein",Very Large Data Bases,SIGMOD Conference,2000,2001
17300,-1.042340,Avi Silberschatz speaks out: on academia versu...,Jeffrey D. Ullman Speaks Out on the Future of ...,Marianne Winslett,Marianne Winslett,ACM SIGMOD Record,SIGMOD Record,2002,2001
5118,-0.858853,Research issues in moving objects databases (t...,DOMINO: Databases fOr MovINg Objects tracking,Ouri Wolfson,"Jutai Zhou, Ouri Wolfson, Sam Chamberlain, A. ...",International Conference on Management of Data,SIGMOD Conference,2000,1999
12977,-0.738860,Special issue on prototypes of deductive datab...,An Introduction to Deductive Database Language...,K. Ramamohanarao,"Kotagiri Ramamohanarao, James Harland",The VLDB Journal &mdash; The International Jou...,VLDB J.,1994,1994


The worst matches are very unlikely to be matches at all. unfourtunately there is no hard line for the match score where the 'matches' stop being actual matches. 
for this example i will take a threshold of 0.80

In [ ]:
thresholdMatches = matched_results.query("best_match_score > 0.80")
print("number of matches above threshold:  {x}".format(x=thresholdMatches['id_right'].count()))

number of matches above threshold:  2178


##Evaluating the accuracy of the matches 
using the DBLP-ACM_perfectMapping.csv as the ground truth

In [ ]:
truePos = 0
falsePos = 0
falseNeg = 0

#find computed match in the actual mappings
for row in thresholdMatches.itertuples():
  found = groundTruth.loc[(groundTruth['idACM'] == row.id_left) & (groundTruth['idDBLP'] == row.id_right)]
  count = found['idDBLP'].count()
  if count == 1: #computed match is in the actual mappings
    truePos += 1
  else: #computed match is not in the actual mappings
    falsePos += 1

#find actual mappings that were not matched
for row in groundTruth.itertuples():
  found = thresholdMatches.loc[(thresholdMatches['id_left'] == row.idACM) & (thresholdMatches['id_right'] == row.idDBLP)]
  count = found['id_right'].count()
  if count == 0: #mapping cannot be found in computed matches
    falseNeg += 1

#Precision = TruePositives / (TruePositives + FalsePositives)
fuzzyPrecision = truePos/(truePos+falsePos)
print("precision: {x}".format(x=fuzzyPrecision))
#Recall = TruePositives / (TruePositives + FalseNegatives)
fuzzyRecall = truePos/(truePos+falseNeg)
print("recall:    {x}".format(x=fuzzyRecall))
#2*((precision*recall)/(precision+recall))
fuzzyF1 = 2*((fuzzyPrecision*fuzzyRecall)/(fuzzyPrecision+fuzzyRecall))
print("F1 score:  {x}".format(x=fuzzyF1))

precision: 0.9940312213039486
recall:    0.9734712230215827
F1 score:  0.9836437982735121


In [ ]:
thresholdMatches.to_excel('fuzzymatcher.xlsx',index=False)

#Record linking using Python Record Linkage Toolkit

In [ ]:
ACM = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv', 
                  index_col='id')
DBLP = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv', 
                   encoding='latin_1', index_col='id')
groundTruth = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/DBLP-ACM_perfectMapping.csv')

##First with full indexing
creating indexer that will compare every record in ACM with every record in DBLP for similarity.

In [ ]:
indexer = recordlinkage.Index()
indexer.full()
candidates = indexer.index(ACM, DBLP)
print('number of candidates: {x}'.format(x=len(candidates)))

number of candidates: 6001104


2294*2616 = just over 6 million pairs so will take a long time (on average almost 7 minutes)

In [ ]:
compare = recordlinkage.Compare()
compare.exact('year', 'year', label='year')
compare.string('title',
            'title',
            threshold=0.85,
            label='title')
compare.string('authors',
            'authors',
            threshold=0.85,
            label='authors')
compare.string('venue',
            'venue',
            threshold=0.85,
            label='venue')
features = compare.compute(candidates, ACM,
                        DBLP)

In [ ]:
features.describe()

,year,title,authors,venue
count,6.001104e+06,6.001104e+06,6.001104e+06,6001104.0
mean,1.001956e-01,3.957605e-04,2.796152e-04,0.0
std,3.002606e-01,1.988980e-02,1.671936e-02,0.0
min,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.0
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.0
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.0
max,1.000000e+00,1.000000e+00,1.000000e+00,0.0


1715 pairs with two or more fields matching

In [ ]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

3.0        669
2.0       1715
1.0     599900
0.0    5398820
dtype: int64

In [ ]:
potential_matches = features[features.sum(axis=1) > 1].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'year':'venue'].sum(axis=1)
potential_matches.head(25)

,id_1,id_2,year,title,authors,venue,Score
0,304586,conf/sigmod/VossenW99,1,1.0,0.0,0.0,2.0
1,304587,conf/sigmod/CruzJ99,1,1.0,0.0,0.0,2.0
2,304589,conf/sigmod/BouguettayaBH99,1,1.0,0.0,0.0,2.0
3,304590,conf/sigmod/BaruGLMPVC99,1,1.0,0.0,0.0,2.0
4,304582,conf/sigmod/BrodskySCE99,1,1.0,0.0,0.0,2.0
5,304583,conf/sigmod/BonnetBCCCHKMMOSS99,1,1.0,0.0,0.0,2.0
6,304584,conf/sigmod/RoussopoulosKS99,1,1.0,0.0,0.0,2.0
7,304585,conf/sigmod/BohlenBD99,1,1.0,0.0,0.0,2.0
8,306112,journals/sigmod/JenningsNF98,1,1.0,0.0,0.0,2.0
9,306115,journals/sigmod/ReichB98,1,1.0,0.0,0.0,2.0


wrangling the data so its easier to review

In [ ]:
ACM['ACM_Lookup'] = ACM[[
    'title', 'authors', 'venue', 'year'
]].apply(lambda x: '_'.join(x.dropna().astype(str)), axis=1)

DBLP['DBLP_Lookup'] = DBLP[[
    'title', 'authors', 'venue', 'year'
]].apply(lambda x: '_'.join(x.dropna().astype(str)), axis=1)

ACM_lookup = ACM[['ACM_Lookup']].reset_index()
ACM_lookup = ACM_lookup.rename(columns={'id': 'id_1'})

DBLP_lookup = DBLP[['DBLP_Lookup']].reset_index()
DBLP_lookup = DBLP_lookup.rename(columns={'id': 'id_2'})

ACM_merge = potential_matches.merge(ACM_lookup, how='left')
final_merge = ACM_merge.merge(DBLP_lookup, how='left')

final_merge.sort_values(by=['id_2', 'Score'], ascending=False)

,id_1,id_2,year,title,authors,venue,Score,ACM_Lookup,DBLP_Lookup
1249,765541,journals/vldb/ZezulaSAR98,1,1.0,0.0,0.0,2.0,Approximate similarity retrieval with M-trees_...,Approximate Similarity Retrieval with M-Trees_...
2254,950494,journals/vldb/YangW03,1,1.0,1.0,0.0,3.0,Incremental computation and maintenance of tem...,Incremental computation and maintenance of tem...
636,615238,journals/vldb/WhiteD95,1,1.0,1.0,0.0,3.0,QuickStore: a high performance mapped object s...,QuickStore: A High Performance Mapped Object S...
1597,223814,journals/vldb/WhiteD95,1,0.0,1.0,0.0,2.0,Implementing crash recovery in QuickStore: a p...,QuickStore: A High Performance Mapped Object S...
1725,191919,journals/vldb/WhiteD95,0,1.0,1.0,0.0,2.0,QuickStore: a high performance mapped object s...,QuickStore: A High Performance Mapped Object S...
...,...,...,...,...,...,...,...,...,...
202,304198,conf/sigmod/AboulnagaC99,1,1.0,0.0,0.0,2.0,Self-tuning histograms: building histograms wi...,Self-tuning Histograms: Building Histograms Wi...
1579,223854,conf/sigmod/AbiteboulCM95,1,1.0,0.0,0.0,2.0,A database interface for file update_Serge Abi...,A Database Interface for File Updates_Tova Mil...
2130,872821,conf/sigmod/AbiteboulBCMM03,1,1.0,0.0,0.0,2.0,Dynamic XML documents with distribution and re...,Dynamic XML documents with distribution and re...
2080,872855,conf/sigmod/AbadiCCCCEGHMRSSTXYZ03,1,1.0,0.0,0.0,2.0,Aurora: a data stream management system_D. Aba...,Aurora: A Data Stream Management System_Nesime...


In [ ]:
final_merge.sort_values(by=['id_2', 'Score'],
                    ascending=False).to_excel('merge_list_full.xlsx',
                                              index=False)

##Evaluating the accuracy of the matches 





In [ ]:
truePos = 0
falsePos = 0
falseNeg = 0

for row in final_merge.itertuples():
  found = groundTruth.loc[(groundTruth['idACM'] == row.id_1) & (groundTruth['idDBLP'] == row.id_2)]
  count = found['idDBLP'].count()
  if count == 1:
    truePos += 1
  else:
    falsePos += 1

for row in groundTruth.itertuples():
  found = final_merge.loc[(final_merge['id_1'] == row.idACM) & (final_merge['id_2'] == row.idDBLP)]
  count = found['id_2'].count()
  if count == 0:
    falseNeg += 1

#Precision = TruePositives / (TruePositives + FalsePositives)
fullPrecision = truePos/(truePos+falsePos)
print("precision: {x}".format(x=fullPrecision))
#Recall = TruePositives / (TruePositives + FalseNegatives)
fullRecall = truePos/(truePos+falseNeg)
print("recall:    {x}".format(x=fullRecall))
#2*((precision*recall)/(precision+recall))
fullF1 = 2*((fullPrecision*fullRecall)/(fullPrecision+fullRecall))
print("F1 score:  {x}".format(x=fullF1))

precision: 0.8942953020134228
recall:    0.9586330935251799
F1 score:  0.9253472222222222


##With blocking

In [ ]:
ACM = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv', 
                  index_col='id')
DBLP = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv', 
                   encoding='latin_1', index_col='id')
groundTruth = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/DBLP-ACM_perfectMapping.csv')

Blocking on year, so only records with the same year are compared. this will reduce the number of records to be compared and, assuming the year between actual matches is always the same, decrease the number of false positives.

In [ ]:
indexer = recordlinkage.Index()
indexer.block(left_on='year', right_on='year')
candidates = indexer.index(ACM, DBLP)
print('number of candidates: {x}'.format(x=len(candidates)))

number of candidates: 601284


this time we are comparing 601,284 possible matches. around 90% less comparisons to be made compared to full indexing so executes quicker. (around 40 seconds)

no need to look for an exact match on year as we are blocking on year (they will always be the same)

In [ ]:
compare = recordlinkage.Compare()
compare.string('title',
            'title',
            threshold=0.85,
            label='title')
compare.string('authors',
            'authors',
            threshold=0.85,
            label='authors')
compare.string('venue',
            'venue',
            threshold=0.85,
            label='venue')
features = compare.compute(candidates, ACM,
                        DBLP)

In [ ]:
features.head()

title  authors  venue
id_1   id_2                                            
304586 journals/sigmod/Mackay99     0.0      0.0    0.0
       conf/sigmod/Keim99           0.0      0.0    0.0
       conf/vldb/ChenCFGJMTW99      0.0      0.0    0.0
       conf/sigmod/BraumandlKK99    0.0      0.0    0.0
       conf/vldb/NinkHR99           0.0      0.0    0.0

In [ ]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

2.0       669
1.0      1676
0.0    598939
dtype: int64

because we blocked on year (theres already a match on year), we only need one more feature to match for two features in total to match (the metric we used in full indexing as a threshold for matched records)

In [ ]:
potential_matches = features[features.sum(axis=1) > 0].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'title':'venue'].sum(axis=1)
potential_matches.head()

,id_1,id_2,title,authors,venue,Score
0,304586,conf/sigmod/VossenW99,1.0,0.0,0.0,1.0
1,304587,conf/sigmod/CruzJ99,1.0,0.0,0.0,1.0
2,304589,conf/sigmod/BouguettayaBH99,1.0,0.0,0.0,1.0
3,304590,conf/sigmod/BaruGLMPVC99,1.0,0.0,0.0,1.0
4,304582,conf/sigmod/BrodskySCE99,1.0,0.0,0.0,1.0


In [ ]:
ACM['ACM_Lookup'] = ACM[[
    'title', 'authors', 'venue', 'year'
]].apply(lambda x: '_'.join(x.dropna().astype(str)), axis=1)

DBLP['DBLP_Lookup'] = DBLP[[
    'title', 'authors', 'venue', 'year'
]].apply(lambda x: '_'.join(x.dropna().astype(str)), axis=1)

ACM_lookup = ACM[['ACM_Lookup']].reset_index()
ACM_lookup = ACM_lookup.rename(columns={'id': 'id_1'})

DBLP_lookup = DBLP[['DBLP_Lookup']].reset_index()
DBLP_lookup = DBLP_lookup.rename(columns={'id': 'id_2'})

ACM_merge = potential_matches.merge(ACM_lookup, how='left')
final_merge = ACM_merge.merge(DBLP_lookup, how='left')

final_merge.sort_values(by=['id_2', 'Score'], ascending=False)

,id_1,id_2,title,authors,venue,Score,ACM_Lookup,DBLP_Lookup
403,765541,journals/vldb/ZezulaSAR98,1.0,0.0,0.0,1.0,Approximate similarity retrieval with M-trees_...,Approximate Similarity Retrieval with M-Trees_...
2276,950494,journals/vldb/YangW03,1.0,1.0,0.0,2.0,Incremental computation and maintenance of tem...,Incremental computation and maintenance of tem...
720,615238,journals/vldb/WhiteD95,1.0,1.0,0.0,2.0,QuickStore: a high performance mapped object s...,QuickStore: A High Performance Mapped Object S...
838,223814,journals/vldb/WhiteD95,0.0,1.0,0.0,1.0,Implementing crash recovery in QuickStore: a p...,QuickStore: A High Performance Mapped Object S...
1913,615187,journals/vldb/WhangKW94,1.0,0.0,0.0,1.0,Dynamic maintenance of data distribution for s...,Dynamic Maintenance of Data Distribution for S...
...,...,...,...,...,...,...,...,...
105,304198,conf/sigmod/AboulnagaC99,1.0,0.0,0.0,1.0,Self-tuning histograms: building histograms wi...,Self-tuning Histograms: Building Histograms Wi...
820,223854,conf/sigmod/AbiteboulCM95,1.0,0.0,0.0,1.0,A database interface for file update_Serge Abi...,A Database Interface for File Updates_Tova Mil...
2204,872821,conf/sigmod/AbiteboulBCMM03,1.0,0.0,0.0,1.0,Dynamic XML documents with distribution and re...,Dynamic XML documents with distribution and re...
2154,872855,conf/sigmod/AbadiCCCCEGHMRSSTXYZ03,1.0,0.0,0.0,1.0,Aurora: a data stream management system_D. Aba...,Aurora: A Data Stream Management System_Nesime...


In [ ]:
final_merge.sort_values(by=['id_2', 'Score'],
                    ascending=False).to_excel('merge_list_block.xlsx',
                                              index=False)

##Evaluating the accuracy of the matches 

In [ ]:
truePos = 0
falsePos = 0
falseNeg = 0

for row in final_merge.itertuples():
  found = groundTruth.loc[(groundTruth['idACM'] == row.id_1) & (groundTruth['idDBLP'] == row.id_2)]
  count = found['idDBLP'].count()
  if count == 1:
    truePos += 1
  else:
    falsePos += 1

for row in groundTruth.itertuples():
  found = final_merge.loc[(final_merge['id_1'] == row.idACM) & (final_merge['id_2'] == row.idDBLP)]
  count = found['id_2'].count()
  if count == 0:
    falseNeg += 1

#Precision = TruePositives / (TruePositives + FalsePositives)
blockPrecision = truePos/(truePos+falsePos)
print("precision: {x}".format(x=blockPrecision))
#Recall = TruePositives / (TruePositives + FalseNegatives)
blockRecall = truePos/(truePos+falseNeg)
print("recall:    {x}".format(x=blockRecall))
#2*((precision*recall)/(precision+recall))
blockF1 = 2*((blockPrecision*blockRecall)/(blockPrecision+blockRecall))
print("F1 score:  {x}".format(x=blockF1))

precision: 0.9091684434968017
recall:    0.9586330935251799
F1 score:  0.9332457868242504


#Comparison


Overall, fuzzymatcher performed the best with an F1 score of 0.98(2 d.p). It has almost perfect precision (0.99) meaning very few incorrect matches and a very high recall meaning few matches missed.

Next is PRLT with blocking with an F1 score of 0.933 (3 d.p). Beating full indexing with an F1 score of 0.925 (3 d.p). These scores are quite close together. However, blocking provided a higher precision (0.91 > 0.89). This means by including the possiblity for records with different years, we introduced more incorrect matches(false positives). Because both methods had the same recall of 0.96 (2 d.p), it could be assumed that both methods missed the same records; therefore, full indexing did not catch any records that were missed by blocking. so it is better to assume that matching records will have the same year.

It should be noted that PRLT does not remove dulplicates (A record could match with multiple records from the other file). This causes the reduced precision due to the increase of incorrectly matched records. Furthermore, because matches are detected in each field individually, i can't give a match pair an overall rating. It cannot determine which match is more likely to be correct between two match pairs if there are the same amount of fields matched. 

In [ ]:
print("Fuzzymatcher scores")
print("")
print("precision: {x}".format(x=fuzzyPrecision))
print("recall:    {x}".format(x=fuzzyRecall))
print("F1 score:  {x}".format(x=fuzzyF1))
print("")
print("")
print("Python record linkage toolkit")
print("")
print("full indexing")
print("precision: {x}".format(x=fullPrecision))
print("recall:    {x}".format(x=fullRecall))
print("F1 score:  {x}".format(x=fullF1))
print("")
print("blocking")
print("precision: {x}".format(x=blockPrecision))
print("recall:    {x}".format(x=blockRecall))
print("F1 score:  {x}".format(x=blockF1))

Fuzzymatcher scores

precision: 0.9940312213039486
recall:    0.9734712230215827
F1 score:  0.9836437982735121


Python record linkage toolkit

full indexing
precision: 0.8942953020134228
recall:    0.9586330935251799
F1 score:  0.9253472222222222

blocking
precision: 0.9091684434968017
recall:    0.9586330935251799
F1 score:  0.9332457868242504


#Better results with fuzzymatcher



In [ ]:
ACM = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv')
DBLP = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv', 
                   encoding='latin_1')
groundTruth = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/DBLP-ACM_perfectMapping.csv')

In [ ]:
# columns from the left dataframe
left_on = ["title", "authors", "venue", "year"]
# columns from the right dataframe
right_on = ["title", "authors", "venue", "year"]

matched_results = fuzzymatcher.fuzzy_left_join(ACM,
                                               DBLP,
                                               left_on,
                                               right_on,
                                               left_id_col='id',
                                               right_id_col='id')

F1 score:0.9836 → 0.9857

I was able to achieve a slightly better F1 score by decreasing the threshold to 0.65. This is trading precision for recall. By doing this, I am hoping to introduce more matches that were missed before than false matches that weren't included before.

In [ ]:
thresholdMatches = matched_results.query("best_match_score > 0.65")
print("number of matches above threshold:  {x}".format(x=thresholdMatches['id_right'].count()))

truePos = 0
falsePos = 0
falseNeg = 0

#find computed match in the actual mappings
for row in thresholdMatches.itertuples():
  found = groundTruth.loc[(groundTruth['idACM'] == row.id_left) & (groundTruth['idDBLP'] == row.id_right)]
  count = found['idDBLP'].count()
  if count == 1: #computed match is in the actual mappings
    truePos += 1
  else: #computed match is not in the actual mappings
    falsePos += 1

#find actual mappings that were not matched
for row in groundTruth.itertuples():
  found = thresholdMatches.loc[(thresholdMatches['id_left'] == row.idACM) & (thresholdMatches['id_right'] == row.idDBLP)]
  count = found['id_right'].count()
  if count == 0: #mapping cannot be found in computed matches
    falseNeg += 1

#Precision = TruePositives / (TruePositives + FalsePositives)
precision = truePos/(truePos+falsePos)
print("precision: {x}".format(x=precision))
#Recall = TruePositives / (TruePositives + FalseNegatives)
recall = truePos/(truePos+falseNeg)
print("recall:    {x}".format(x=recall))
#2*((precision*recall)/(precision+recall))
F1 = 2*((precision*recall)/(precision+recall))
print("F1 score:  {x}".format(x=F1))

thresholdMatches.to_excel('improvedfuzzymatcher.xlsx',index=False)

number of matches above threshold:  2193
precision: 0.9927040583675331
recall:    0.9788669064748201
F1 score:  0.9857369255150554
